In [8]:
from importlib import reload
import os
from toolz import pipe as p

import scrapDataHelpers

In [9]:
ls image_data/

gossiping/  laughing/     pullinghair/  quarrel/   stabbing/
isolation/  nonbullying/  punching/     slapping/  strangle/


In [13]:
reload(scrapDataHelpers)

classes = ('laughing', 'pullinghair', 'gossiping')
scrapDataHelpers.makeScrapData(classes, n_train=80, n_val=20)

In [20]:
xs = ('train', 'val')

folders = [os.path.join('scrap_data', x, c) for x in xs
          for c in classes]

{f.replace('scrap_data/', ''): p(f, os.listdir, len) for f in folders}

{'train/laughing': 13,
 'train/pullinghair': 26,
 'train/gossiping': 41,
 'val/laughing': 6,
 'val/pullinghair': 5,
 'val/gossiping': 9}